**For WandB Logging**

In [9]:
!pip install wandb

**Initiating WandB**

In [15]:
import wandb

wandb.init(project="research-1288-sam", name="research-1288-sam-run")

**Mounting Google Drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Installing Ultralytics for YOLOv8**

In [4]:
!pip install ultralytics

**Using SAM's "auto-annotate" Tool to Generate Train Set's Labels Again for Experiment**

In [17]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data="/content/drive/MyDrive/research/1288/train/images", det_model="/content/drive/MyDrive/best.pt", sam_model="sam_b.pt", output_dir="/content/drive/MyDrive/research/1288/train/labels")


image 1/1240 /content/drive/MyDrive/research/1288/train/images/intraoral_lower-1-_JPEG.rf.25679f1ac65a7dcf357a249eb9f23277.jpg: 480x640 1 31, 1 32, 1 33, 1 34, 1 35, 1 36, 1 37, 1 41, 1 42, 1 43, 1 44, 1 45, 1 46, 1 staining or visible changes without cavitation, 1 tongue, 56.8ms
image 2/1240 /content/drive/MyDrive/research/1288/train/images/intraoral_lower-1-_JPEG.rf.2e1183d7d856172616d6fe5be00910ef.jpg: 480x640 1 31, 1 32, 1 34, 1 35, 2 36s, 1 37, 1 41, 1 42, 1 43, 1 44, 1 45, 1 46, 1 staining or visible changes without cavitation, 1 tongue, 8.8ms
image 3/1240 /content/drive/MyDrive/research/1288/train/images/intraoral_lower-1-_JPEG.rf.4ba375ab327225ba617374286518cf7e.jpg: 480x640 1 31, 1 32, 1 33, 1 34, 1 35, 1 36, 1 37, 1 41, 1 42, 1 43, 1 44, 1 45, 1 46, 1 staining or visible changes without cavitation, 1 tongue, 10.8ms
image 4/1240 /content/drive/MyDrive/research/1288/train/images/intraoral_lower-1-_JPEG.rf.4bd3920f30703d4ecc26ae0543ee8533.jpg: 480x640 1 31, 1 32, 1 33, 1 34, 1 

**Training with YOLOv8 Small Segmentation Model after Deleting the Train Set's Labels and Placing SAM-Generated Labels There**

In [18]:
import os
os.path.exists("/content/drive/MyDrive/research/1288/data.yaml")

True

In [19]:
from ultralytics import YOLO

# Load YOLOv8n-seg model
model = YOLO("/content/drive/MyDrive/yolov8s-seg.pt")  # pretrained on COCO

# Train the model
results = model.train(
    data="/content/drive/MyDrive/research/1288/data.yaml",
    epochs=35,
    imgsz=640,
    batch=16,
    project="runs/train/1288-sam",
    name="1288-sam-run",
    exist_ok=True,
    task="segment",
    plots=True,
    auto_augment=False,
    augment=False,
    optimizer="SGD",       # SGD optimizer
    lr0=0.01,              # Initial learning rate
    lrf=0.001,             # Final learning rate
    patience=7,            # Patience for early stopping
    momentum=0.937,        # Momentum
    seed=42,
    save=True,
    val=True
)

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=False, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/research/1288/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=35, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=1288-sam-run, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, pati

**WandB Logging**

In [20]:
import wandb
import os

RUN_DIR = "/content/runs/train/1288-sam/1288-sam-run"

wandb.init(project="research-1288-sam", entity="adityadatta30-north-south-university", name="research-1288-sam-results")

# Upload weights
wandb.save(os.path.join(RUN_DIR, "weights/best.pt"))
wandb.save(os.path.join(RUN_DIR, "weights/last.pt"))

# Upload CSV metrics
wandb.save(os.path.join(RUN_DIR, "results.csv"))

# Upload all PNG/JPG images
for file in os.listdir(RUN_DIR):
    if file.endswith((".png", ".jpg", ".jpeg")):
        wandb.log({file: wandb.Image(os.path.join(RUN_DIR, file))})

wandb.finish()


wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.
wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.
wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


In [28]:
!cp -r /content/runs/train/1288-sam/1288-sam-run /content/drive/MyDrive/runs/1288-sam/1288-sam-run